In [1]:
from src.data import MovieLens20MDataLoader
import pandas as pd
from tqdm import tqdm_notebook as tqdm
from sklearn.metrics.pairwise import cosine_similarity 
import sys
import numpy as np

sys.path

['/app/notebook',
 '/usr/local/lib/python310.zip',
 '/usr/local/lib/python3.10',
 '/usr/local/lib/python3.10/lib-dynload',
 '',
 '/usr/local/lib/python3.10/site-packages',
 '/app']

In [2]:
ml_loader = MovieLens20MDataLoader()
ml_data = ml_loader.load()

file is found. loading...


In [3]:
_movies = ml_data.get('movies')
_movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
27273,131254,Kein Bund für's Leben (2007),Comedy
27274,131256,"Feuer, Eis & Dosenbier (2002)",Comedy
27275,131258,The Pirates (2014),Adventure
27276,131260,Rentun Ruusu (2001),(no genres listed)


In [9]:
genre_features = ['Action', "Adventure", "Animation", "Children", "Comedy", "Crime", "Documentary", "Drama", "Fantasy",
               "Film-Noir", "Horror", "Musical", "Mystery", "Romance", "Sci-Fi", "Thriller", "War", "Western",
               "(no genres listed)", "IMAX"]
genres = pd.DataFrame([], columns=genre_features)
movies = pd.concat([_movies, genres], axis=1)

for i, row in enumerate(tqdm(movies.itertuples(), total=len(movies))):
    gs = row.genres.split('|')
    for g in gs:
        movies.loc[movies['movieId'] == row.movieId, g] = 1

movies = movies.fillna(0)
movies

/tmp/ipykernel_1258/1195162325.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i, row in enumerate(tqdm(movies.itertuples(), total=len(movies))):


  0%|          | 0/27278 [00:00<?, ?it/s]

,movieId,title,genres,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,...,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,(no genres listed),IMAX
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,0,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji (1995),Adventure|Children|Fantasy,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,Grumpier Old Men (1995),Comedy|Romance,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0
4,5,Father of the Bride Part II (1995),Comedy,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27273,131254,Kein Bund für's Leben (2007),Comedy,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
27274,131256,"Feuer, Eis & Dosenbier (2002)",Comedy,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
27275,131258,The Pirates (2014),Adventure,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27276,131260,Rentun Ruusu (2001),(no genres listed),0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [10]:
simirality_feature = genre_features
simirality_feature.remove("(no genres listed)")
print(genre_features)
print(simirality_feature)

item_ids = movies['movieId'].tolist()
item_matrix = pd.DataFrame(
    cosine_similarity(movies[simirality_feature], movies[simirality_feature]),
    columns = item_ids, index = item_ids
)

item_matrix.head()

['Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western', 'IMAX']
['Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western', 'IMAX']


,1,2,3,4,5,6,7,8,9,10,...,131241,131243,131248,131250,131252,131254,131256,131258,131260,131262
1,1.000000,0.774597,0.316228,0.258199,0.447214,0.0,0.316228,0.632456,0.0,0.258199,...,0.316228,0.632456,1.000000,0.447214,0.316228,0.447214,0.447214,0.447214,0.0,0.516398
2,0.774597,1.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.816497,0.0,0.333333,...,0.000000,0.000000,0.774597,0.000000,0.000000,0.000000,0.000000,0.577350,0.0,0.666667
3,0.316228,0.000000,1.000000,0.816497,0.707107,0.0,1.000000,0.000000,0.0,0.000000,...,1.000000,0.500000,0.316228,0.707107,0.500000,0.707107,0.707107,0.000000,0.0,0.000000
4,0.258199,0.000000,0.816497,1.000000,0.577350,0.0,0.816497,0.000000,0.0,0.000000,...,0.816497,0.408248,0.258199,0.577350,0.408248,0.577350,0.577350,0.000000,0.0,0.000000
5,0.447214,0.000000,0.707107,0.577350,1.000000,0.0,0.707107,0.000000,0.0,0.000000,...,0.707107,0.707107,0.447214,1.000000,0.707107,1.000000,1.000000,0.000000,0.0,0.000000


In [20]:
recommend_item_id = 1

a = item_matrix[recommend_item_id].sort_values(ascending=False).head(80)
a1 = list(zip(a.index, a))

result = pd.DataFrame()
for ai in a1:
    result.loc[ai[0], 'movieId'] = ai[0]
    result.loc[ai[0], 'original_item'] = ai[0] == recommend_item_id
    result.loc[ai[0], 'title'] = movies.loc[_movies['movieId'] == ai[0], 'title'].values[0]
    result.loc[ai[0], 'genres'] = movies.loc[_movies['movieId'] == ai[0], 'genres'].values[0]
    result.loc[ai[0], 'similarity'] = ai[1]
    
result

,movieId,original_item,title,genres,similarity
1,1.0,True,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1.000000
115875,115875.0,False,Toy Story Toons: Hawaiian Vacation (2011),Adventure|Animation|Children|Comedy|Fantasy,1.000000
53121,53121.0,False,Shrek the Third (2007),Adventure|Animation|Children|Comedy|Fantasy,1.000000
33463,33463.0,False,DuckTales: The Movie - Treasure of the Lost La...,Adventure|Animation|Children|Comedy|Fantasy,1.000000
65577,65577.0,False,"Tale of Despereaux, The (2008)",Adventure|Animation|Children|Comedy|Fantasy,1.000000
...,...,...,...,...,...
27619,27619.0,False,"Lion King 1½, The (2004)",Adventure|Animation|Children|Comedy,0.894427
26565,26565.0,False,Asterix in Britain (Astérix chez les Bretons) ...,Adventure|Animation|Children|Comedy,0.894427
26529,26529.0,False,Asterix vs. Caesar (Astérix et la surprise de ...,Adventure|Animation|Children|Comedy,0.894427
8974,8974.0,False,"SpongeBob SquarePants Movie, The (2004)",Adventure|Animation|Children|Comedy,0.894427
